In [1]:
import pandas as pd
import numpy as np

In [2]:
#loading the dataset
dataset = pd.read_csv('drive/MyDrive/NLP Project data/Floods_combined.csv')   #Link to original dataset

In [3]:
dataset

,Unnamed: 0,id,tweets,label
0,8074,8074,Our credit taking operation on social media is...,0
1,4207,4207,Israeli startup warned Chiles citizens of tsun...,0
2,3127,3127,My thoughts are with my friends neighbors in ...,0
3,5756,5756,Canine Disaster Search Teams enroute to Nepal ...,0
4,3958,3958,NEW Tsunami WARNING for Chile Peru via Magni...,0
...,...,...,...,...
17375,23584,23584,Pungovt claims CMSS took helicopter flightsSo...,0
17376,23585,23585,Pakistan Flood Sinks Boat Carrying Wedding Par...,1
17377,23586,23586,Jammu and Kashmir Floods Yasin Malik Creates H...,0
17378,23587,23587,Please use Kashmir Flood hashtag only if u nee...,0


In [4]:
dataset['label'].value_counts()

0    12743
1     4637
Name: label, dtype: int64

In [ ]:
positive = ['injured_or_dead_people', 'missing_trapped_or_found_people', 'displaced_people_and_evacuations', 'infrastructure_and_utilities_damage']
negative = ['donation_needs_or_offers_or_volunteering_services', 'caution_and_advice', 'sympathy_and_emotional_support', 'other_useful_information', 'not_related_or_irrelevant']


for index, row in dataset.iterrows():
  if row['label'] in positive:
    row['label']=1
  elif row['label'] in negative:
    row['label'] = 0

In [ ]:
dataset['label'].value_counts()

0    2095
1    1494
Name: label, dtype: int64

In [ ]:
# #Reading OOV words and forming a dictionary
def OOV_dict(path_to_OOV):
  import pandas as pd
  OOV = pd.read_csv (path_to_OOV,sep='\t',header=None)
  dictionary_OOV={}
  for row in OOV.iterrows():
    dictionary_OOV[row[1][0]]=row[1][1]
  return dictionary_OOV

oov_dict = OOV_dict("https://0f41bdd6-d0a9-458e-937c-838e849bb6f4.usrfiles.com/ugd/0f41bd_89f0c122dbf6433183f7add90a28ec33.csv")

In [ ]:
pp_df = pd.DataFrame()
pp_df['tweets'] = dataset.tweet_text

In [ ]:
pp_df.head()

,tweets
0,Prayers for Cabo: Hurricane Odile Roars Throug...
1,Sammy Hagar's Home Damaged in Hurricane: Sammy...
2,"Residents, Tourists Sent to Shelters as Hurric..."
3,Our thoughts go out to all of our friends in L...
4,Does anyone have information on emergency serv...


In [ ]:
# Replacing OOV words
for index, row in pp_df.iterrows():
  sent_list = row['tweets'].split()
  for i in range(len(sent_list)):
    if sent_list[i] in oov_dict.keys():
      sent_list[i] = oov_dict[sent_list[i]]
  updated_sent = " "
  updated_sent = updated_sent.join(sent_list)
  row['tweets'] = updated_sent

,tweets
0,Prayers for Cabo: Hurricane Odile Roars Throug...
1,Sammy Hagar's Home Damaged in Hurricane: Sammy...
2,"Residents, Tourists Sent to Shelters as Hurric..."
3,Our thoughts go out to all of our friends in L...
4,Does anyone have information on emergency serv...


In [ ]:
# Removing the URLs
import re
# https or http URLs
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r'http?:\/\/\S+', '', x))

# www. URLs
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

# removing [video] and [Video]
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r"\[video\]", '', x))
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r"\[Video\]", '', x))

In [ ]:
# Removing HTML Reference Characters
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))

#Removing @ mentions
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r'(\s)@\w+', r'\1',x))


In [ ]:
#Removing # symbols
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r"#",'',x))

In [ ]:
#Removing RT symbols
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r"RT",'',x))

#for punctuation marks
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: re.sub(r'[.,`"-?:!;【】|@♨😔…°→”“😂☀❤\[\]▼💡’‘🙈👏~🙏¦»♦♥•😒✔—👌💝¤😩😟😁😪🎉😢💗↑💖🍃☯╚►➡✋☇⬅✡£😚♫Ü🌎🙌✞🙉😜👊✨🎊►💜↓🎋🎉😋😭💃♛😱☺➤✪￼_😦💕✌️😃💙🌞😍😊😅🎵🙍🙇😞💔😖♡😳★😕😨📚📖✏️😛👼]','',x))

#Striping the white spaces
pp_df['tweets'] = pp_df['tweets'].apply(lambda x: x.strip())

In [ ]:
#Adding labels to preprocessed Tweets
pp_df['label'] = dataset['label']

In [ ]:
pp_df.to_csv('pp_tweets.csv')